In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using Theano backend.


## This notebook is created from the basic GAN example @ https://github.com/eriklindernoren/Keras-GAN

## To apply to different data, change the data in the train function, some great datasets to use for learning can be found @ https://www.tensorflow.org/api_docs/python/tf/keras/datasets

## Make sure you create a folder in the same directory as this file named "images" This is where the generated images will be stored

<img src="https://user-images.githubusercontent.com/33185528/57170499-f277ad80-6dd2-11e9-8be7-9866c932d343.png">

### This first block creates the GAN class. It defines the basic parameters of the GAN model. Rows & cols (our features) are defined to 28 as the images from mnist are by default 28x28. The descriminator is pre-trained in this example and will not improve over time; for this reason we set trainable = false. 

In [3]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

## This builds the generator neural network. The generator is responsible for generating images using guidance from the descriminator. The generator seeks to minimize the accuracy of the discriminator. 

In [12]:
class GAN(GAN):
    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

## This builds the discriminator neural network. The descriminator is responsible for classifying generated images as real or fake. 

In [5]:
class GAN(GAN): 
    def build_discriminator(self):

            model = Sequential()

            model.add(Flatten(input_shape=self.img_shape))
            model.add(Dense(512))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dense(256))
            model.add(LeakyReLU(alpha=0.2))
            model.add(Dense(1, activation='sigmoid'))
            model.summary()

            img = Input(shape=self.img_shape)
            validity = model(img)

            return Model(img, validity)

## This function handles training. This is where you load your dataset, noise for input to the generator, etc.

In [14]:
class GAN(GAN):
    def train(self, epochs, batch_size=128, sample_interval=50):

            # Load the dataset
            (X_train, _), (_, _) = mnist.load_data()

            # Rescale -1 to 1
            X_train = X_train / 127.5 - 1.
            X_train = np.expand_dims(X_train, axis=3)

            # Adversarial ground truths
            valid = np.ones((batch_size, 1))
            fake = np.zeros((batch_size, 1))

            for epoch in range(epochs):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Train the generator (to have the discriminator label samples as valid)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if epoch % sample_interval == 0:
                    self.sample_images(epoch)

In [15]:
class GAN(GAN):
    def sample_images(self, epoch):
            r, c = 5, 5
            noise = np.random.normal(0, 1, (r * c, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(r, c)
            cnt = 0
            for i in range(r):
                for j in range(c):
                    axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                    axs[i,j].axis('off')
                    cnt += 1
            fig.savefig("images/%d.png" % epoch)
            plt.close()

In [8]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\Posey\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Posey\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.579616, acc.: 50.00%] [G loss: 0.497964]


C:\Users\Posey\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.377045, acc.: 62.50%] [G loss: 0.570088]
2 [D loss: 0.353287, acc.: 76.56%] [G loss: 0.565742]
3 [D loss: 0.334578, acc.: 79.69%] [G loss: 0.726999]
4 [D loss: 0.308887, acc.: 89.06%] [G loss: 0.827573]
5 [D loss: 0.282112, acc.: 93.75%] [G loss: 0.953433]
6 [D loss: 0.232820, acc.: 95.31%] [G loss: 1.090133]
7 [D loss: 0.206813, acc.: 100.00%] [G loss: 1.345829]
8 [D loss: 0.159510, acc.: 98.44%] [G loss: 1.508582]
9 [D loss: 0.137156, acc.: 100.00%] [G loss: 1.739894]
10 [D loss: 0.115480, acc.: 100.00%] [G loss: 1.939943]
11 [D loss: 0.090379, acc.: 100.00%] [G loss: 2.026911]
12 [D loss: 0.065129, acc.: 100.00%] [G loss: 2.113215]
13 [D loss: 0.065259, acc.: 100.00%] [G loss: 2.238605]
14 [D loss: 0.081174, acc.: 100.00%] [G loss: 2.394993]
15 [D loss: 0.063582, acc.: 100.00%] [G loss: 2.577155]
16 [D loss: 0.056240, acc.: 100.00%] [G loss: 2.546870]
17 [D loss: 0.044392, acc.: 100.00%] [G loss: 2.772843]
18 [D loss: 0.047706, acc.: 100.00%] [G loss: 2.665234]
19 [D lo

...

29400 [D loss: 0.635846, acc.: 60.94%] [G loss: 1.072906]
29401 [D loss: 0.684910, acc.: 54.69%] [G loss: 1.059457]
29402 [D loss: 0.710352, acc.: 64.06%] [G loss: 1.101763]
29403 [D loss: 0.678017, acc.: 64.06%] [G loss: 1.118758]
29404 [D loss: 0.618283, acc.: 68.75%] [G loss: 0.995848]
29405 [D loss: 0.515727, acc.: 84.38%] [G loss: 0.965274]
29406 [D loss: 0.631741, acc.: 50.00%] [G loss: 0.998284]
29407 [D loss: 0.605572, acc.: 68.75%] [G loss: 1.013288]
29408 [D loss: 0.605322, acc.: 67.19%] [G loss: 1.042634]
29409 [D loss: 0.612952, acc.: 64.06%] [G loss: 1.035525]
29410 [D loss: 0.635176, acc.: 64.06%] [G loss: 0.974028]
29411 [D loss: 0.682650, acc.: 60.94%] [G loss: 1.010892]
29412 [D loss: 0.594450, acc.: 62.50%] [G loss: 1.118652]
29413 [D loss: 0.652519, acc.: 62.50%] [G loss: 1.027285]
29414 [D loss: 0.619005, acc.: 71.88%] [G loss: 0.962165]
29415 [D loss: 0.642893, acc.: 60.94%] [G loss: 1.061032]
29416 [D loss: 0.591777, acc.: 73.44%] [G loss: 0.952902]
29417 [D loss:

29543 [D loss: 0.604498, acc.: 67.19%] [G loss: 1.170117]
29544 [D loss: 0.595584, acc.: 73.44%] [G loss: 0.920475]
29545 [D loss: 0.715547, acc.: 57.81%] [G loss: 1.112654]
29546 [D loss: 0.641503, acc.: 64.06%] [G loss: 1.149749]
29547 [D loss: 0.615117, acc.: 68.75%] [G loss: 1.104376]
29548 [D loss: 0.607844, acc.: 65.62%] [G loss: 1.066087]
29549 [D loss: 0.623086, acc.: 59.38%] [G loss: 1.124325]
29550 [D loss: 0.692127, acc.: 65.62%] [G loss: 1.062183]
29551 [D loss: 0.598088, acc.: 62.50%] [G loss: 0.867555]
29552 [D loss: 0.635298, acc.: 64.06%] [G loss: 1.039880]
29553 [D loss: 0.549985, acc.: 70.31%] [G loss: 1.023663]
29554 [D loss: 0.633943, acc.: 67.19%] [G loss: 1.095411]
29555 [D loss: 0.638506, acc.: 57.81%] [G loss: 0.949681]
29556 [D loss: 0.729970, acc.: 56.25%] [G loss: 0.998510]
29557 [D loss: 0.593674, acc.: 70.31%] [G loss: 1.006886]
29558 [D loss: 0.586497, acc.: 76.56%] [G loss: 1.035491]
29559 [D loss: 0.666662, acc.: 59.38%] [G loss: 1.067540]
29560 [D loss:

29685 [D loss: 0.616974, acc.: 68.75%] [G loss: 0.999737]
29686 [D loss: 0.595605, acc.: 68.75%] [G loss: 0.972805]
29687 [D loss: 0.598071, acc.: 70.31%] [G loss: 1.117893]
29688 [D loss: 0.649873, acc.: 59.38%] [G loss: 1.077932]
29689 [D loss: 0.579416, acc.: 70.31%] [G loss: 0.935042]
29690 [D loss: 0.622705, acc.: 64.06%] [G loss: 1.060686]
29691 [D loss: 0.610229, acc.: 67.19%] [G loss: 1.033479]
29692 [D loss: 0.667585, acc.: 57.81%] [G loss: 0.977730]
29693 [D loss: 0.527468, acc.: 78.12%] [G loss: 0.906968]
29694 [D loss: 0.693208, acc.: 59.38%] [G loss: 1.120484]
29695 [D loss: 0.655123, acc.: 59.38%] [G loss: 1.045612]
29696 [D loss: 0.614588, acc.: 68.75%] [G loss: 1.236458]
29697 [D loss: 0.619922, acc.: 64.06%] [G loss: 0.983961]
29698 [D loss: 0.629874, acc.: 64.06%] [G loss: 1.112406]
29699 [D loss: 0.630794, acc.: 67.19%] [G loss: 0.989583]
29700 [D loss: 0.701114, acc.: 57.81%] [G loss: 0.957007]
29701 [D loss: 0.566811, acc.: 67.19%] [G loss: 1.011478]
29702 [D loss:

29828 [D loss: 0.537299, acc.: 76.56%] [G loss: 0.984561]
29829 [D loss: 0.598490, acc.: 70.31%] [G loss: 1.076880]
29830 [D loss: 0.575420, acc.: 67.19%] [G loss: 1.034708]
29831 [D loss: 0.620790, acc.: 64.06%] [G loss: 0.954973]
29832 [D loss: 0.631336, acc.: 67.19%] [G loss: 1.096271]
29833 [D loss: 0.650514, acc.: 60.94%] [G loss: 0.935214]
29834 [D loss: 0.636067, acc.: 62.50%] [G loss: 1.047187]
29835 [D loss: 0.628874, acc.: 60.94%] [G loss: 0.946085]
29836 [D loss: 0.690758, acc.: 59.38%] [G loss: 0.993958]
29837 [D loss: 0.571785, acc.: 67.19%] [G loss: 1.010095]
29838 [D loss: 0.600337, acc.: 62.50%] [G loss: 1.179675]
29839 [D loss: 0.569317, acc.: 71.88%] [G loss: 1.127485]
29840 [D loss: 0.702615, acc.: 59.38%] [G loss: 1.080065]
29841 [D loss: 0.574537, acc.: 75.00%] [G loss: 0.917978]
29842 [D loss: 0.699271, acc.: 57.81%] [G loss: 1.026795]
29843 [D loss: 0.622928, acc.: 65.62%] [G loss: 1.066658]
29844 [D loss: 0.743533, acc.: 43.75%] [G loss: 0.983731]
29845 [D loss:

29970 [D loss: 0.578916, acc.: 71.88%] [G loss: 1.055789]
29971 [D loss: 0.563789, acc.: 71.88%] [G loss: 1.047932]
29972 [D loss: 0.619275, acc.: 64.06%] [G loss: 1.066168]
29973 [D loss: 0.680793, acc.: 57.81%] [G loss: 1.036078]
29974 [D loss: 0.651065, acc.: 60.94%] [G loss: 1.004950]
29975 [D loss: 0.603427, acc.: 70.31%] [G loss: 0.907654]
29976 [D loss: 0.664871, acc.: 59.38%] [G loss: 1.036335]
29977 [D loss: 0.654832, acc.: 60.94%] [G loss: 1.075196]
29978 [D loss: 0.717206, acc.: 60.94%] [G loss: 1.091153]
29979 [D loss: 0.672277, acc.: 68.75%] [G loss: 0.980873]
29980 [D loss: 0.604607, acc.: 67.19%] [G loss: 1.091540]
29981 [D loss: 0.555223, acc.: 71.88%] [G loss: 1.078187]
29982 [D loss: 0.653383, acc.: 64.06%] [G loss: 1.047262]
29983 [D loss: 0.697539, acc.: 56.25%] [G loss: 1.020776]
29984 [D loss: 0.631934, acc.: 62.50%] [G loss: 1.048640]
29985 [D loss: 0.644493, acc.: 65.62%] [G loss: 1.051774]
29986 [D loss: 0.647581, acc.: 65.62%] [G loss: 0.964838]
29987 [D loss: